# ResNet50
Link: https://tykimos.github.io/warehouse/2018-3-19-ISS_Python_Deep_Learning_Keras_with_Blocks_jmj.pdf

API: https://keras.io/api/applications/

In [32]:
import argparse
import sys

# Tensorflow ans tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from skimage.transform import resize

# keras libraries
from keras import models
from keras.models import load_model, Sequential
from keras import layers
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

# resnet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np




print(tf.__version__)

1.14.0


In [33]:
(X_train_org, y_train_org), (X_test_org, y_test_org) = tf.keras.datasets.mnist.load_data()
print(y_train_org[0])

X_train_224 = resize(X_train_org, (224,224))
X_test_224 = resize(X_test_org, (224,224))

# X_train = X_train_org.astype(np.float32).reshape(-1, 28*28) / 255.0 # image ### Image
# X_test = X_test_org.astype(np.float32).reshape(-1, 28*28) / 255.0 # image
X_train = X_train_224.astype(np.float32).reshape(-1, 224*224) / 255.0 # image ### Image
X_test = X_test_224.astype(np.float32).reshape(-1, 224*224) / 255.0 # image




y_train = tf.keras.utils.to_categorical(y_train_org) ### label (class vector(integer) => binary class matrix): 범주형 인코딩
y_test = tf.keras.utils.to_categorical(y_test_org) 
print(y_train[0])
y_train = y_train.astype(np.int32) # label => 
y_test = y_test.astype(np.int32)
print(y_train[0])

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0 0 0 0 0 1 0 0 0 0]


In [34]:
# model
model = ResNet50(weights=None)

In [39]:
# 3. 모델 학습과정 설정하기
# model.compile(loss='categorical_crossentropy', optimizer='adam',
#               metrics=['accuracy']) # 최적화 알고리즘 설정

model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [40]:
# 4. 모델 학습시키기
# model.fit_generator(
#  train_generator, # 훈련셋 지정
#  steps_per_epoch=200, # 총 훈련셋 수 / 배치 사이즈 (= 1000/50)
#  epochs=150) # 전체 훈련셋 학습 반복 횟수 지정
model.fit(X_train, y_train, epochs=5, batch_size=128)

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (28, 50176)

In [ ]:
# 5. 모델 평가하기
score=model.evaluate_generator(test_generator, steps=3) 